In [1]:
import pandas as pd
import psycopg2 as pg

# Import SQL Alchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Date, ForeignKey

from sqlalchemy.orm import relationship, scoped_session, sessionmaker

In [22]:
# Study data files
departments = pd.read_csv('../data/departments.csv')
dept_emp = pd.read_csv('../data/dept_emp.csv')
dept_manager = pd.read_csv('../data/dept_manager.csv')
employees = pd.read_csv('../data/employees.csv')
salaries = pd.read_csv('../data/salaries.csv')
titles = pd.read_csv('../data/titles.csv')


In [23]:
employees.count()

emp_no          300024
emp_title_id    300024
birth_date      300024
first_name      300024
last_name       300024
sex             300024
hire_date       300024
dtype: int64

In [24]:
# Combine the data into a single dataset
employee_merge = pd.merge(employees,salaries, on = "emp_no", how = 'left')
employee_merge.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date,salary
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,4/28/1990,40000
1,475053,e0002,11/18/1954,Byong,Delgrande,F,9/7/1991,53422
2,57444,e0002,1/30/1958,Berry,Babb,F,3/21/1992,48973
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,11/26/1987,40000
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1/18/1991,40000


In [25]:
employee_merge.count()

emp_no          300024
emp_title_id    300024
birth_date      300024
first_name      300024
last_name       300024
sex             300024
hire_date       300024
salary          300024
dtype: int64

In [26]:
employee_dept = pd.merge(employee_merge,dept_emp, on = "emp_no", how = 'left')
employee_dept.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date,salary,dept_no
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,4/28/1990,40000,d002
1,475053,e0002,11/18/1954,Byong,Delgrande,F,9/7/1991,53422,d004
2,57444,e0002,1/30/1958,Berry,Babb,F,3/21/1992,48973,d004
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,11/26/1987,40000,d003
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1/18/1991,40000,d006


In [27]:
employee_dept.count()

emp_no          331603
emp_title_id    331603
birth_date      331603
first_name      331603
last_name       331603
sex             331603
hire_date       331603
salary          331603
dept_no         331603
dtype: int64

In [29]:
employee_departments = pd.merge(employee_dept,departments, on = 'dept_no', how = 'left')
employee_departments.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date,salary,dept_no,dept_name
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,4/28/1990,40000,d002,Finance
1,475053,e0002,11/18/1954,Byong,Delgrande,F,9/7/1991,53422,d004,Production
2,57444,e0002,1/30/1958,Berry,Babb,F,3/21/1992,48973,d004,Production
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,11/26/1987,40000,d003,Human Resources
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1/18/1991,40000,d006,Quality Management


In [30]:
employee_manager = pd.merge(employee_departments,dept_manager, on = 'emp_no', how = 'left')
employee_manager.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date,salary,dept_no_x,dept_name,dept_no_y
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,4/28/1990,40000,d002,Finance,NaN
1,475053,e0002,11/18/1954,Byong,Delgrande,F,9/7/1991,53422,d004,Production,NaN
2,57444,e0002,1/30/1958,Berry,Babb,F,3/21/1992,48973,d004,Production,NaN
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,11/26/1987,40000,d003,Human Resources,NaN
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1/18/1991,40000,d006,Quality Management,NaN


In [31]:
employee_df = employee_manager.rename(columns={'emp_title_id':'title_id', 'dept_no_x': 'dept_no'})
employee_df = employee_df.drop(columns = ['dept_no_y'])
employee_df.head()

,emp_no,title_id,birth_date,first_name,last_name,sex,hire_date,salary,dept_no,dept_name
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,4/28/1990,40000,d002,Finance
1,475053,e0002,11/18/1954,Byong,Delgrande,F,9/7/1991,53422,d004,Production
2,57444,e0002,1/30/1958,Berry,Babb,F,3/21/1992,48973,d004,Production
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,11/26/1987,40000,d003,Human Resources
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1/18/1991,40000,d006,Quality Management


In [19]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [20]:
employee_df.to_sql('employees', con=engine)
engine.execute("SELECT * FROM employees").fetchall()

[(0, 473302, 's0001', '7/25/1953', 'Hideyuki', 'Zallocco', 'M', '4/28/1990', 40000, 'd002', 'Finance'),
 (1, 461591, 's0002', '11/17/1964', 'Xudong', 'Samarati', 'M', '11/13/1985', 40000, 'd002', 'Finance'),
 (2, 48085, 's0001', '1/19/1964', 'Venkatesan', 'Gilg', 'M', '6/28/1993', 63016, 'd002', 'Finance'),
 (3, 424270, 's0001', '1/15/1963', 'Kellyn', 'Yoshizawa', 'F', '5/8/1995', 60678, 'd002', 'Finance'),
 (4, 424270, 's0001', '1/15/1963', 'Kellyn', 'Yoshizawa', 'F', '5/8/1995', 60678, 'd007', 'Sales'),
 (5, 37473, 's0001', '3/6/1952', 'Prasadram', 'Valette', 'M', '2/16/1994', 55963, 'd002', 'Finance'),
 (6, 293744, 's0001', '4/29/1952', 'Kolar', 'Carrere', 'M', '6/30/1987', 40000, 'd002', 'Finance'),
 (7, 293744, 's0001', '4/29/1952', 'Kolar', 'Carrere', 'M', '6/30/1987', 40000, 'd003', 'Human Resources'),
 (8, 92149, 's0001', '3/19/1957', 'Serenella', 'Kamber', 'F', '9/14/1988', 69585, 'd002', 'Finance'),
 (9, 14097, 's0001', '3/18/1963', 'Khalil', 'Gniady', 'M', '12/4/1990', 70604

In [ ]:
# Create classes with primary keys and foreign keys

class Department (Base):
    __tablename__ = 'departments'
    __table_args__ = {'extend_existing': True}
    dept_no = Column(String, primary_key = True)
    dept_name = Column(String)
    
class Title (Base):
    __tablename__ = 'titles'
    __table_args__ = {'extend_existing': True}
    title_id = Column(String, primary_key = True)
    title = Column(String)
    
class Employee(Base):
    __tablename__ = 'employees'
    __table_args__ = {'extend_existing': True}
    emp_no = Column(Integer, primary_key = True)
    emp_title_id = Column(String, ForeignKey('titles.title_id'))
    birth_date = Column(Date)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(String)
    hire_date = Column(Date)
    #title = relationship('Title', back_populates = 'employees')
    title_employees = relationship('Title', foreign_keys = 'Employee.emp_title_id')
#Title.employees = relationship('Employee', order_by = Employee.emp_title_id, back_populates = 'title')

class Dept_employee (Base):
    __tablename__ = 'dept_emp'
    __table_args__ = {'extend_existing': True}
    emp_no = Column(Integer, ForeignKey('employees.emp_no'),primary_key = True)
    dept_no = Column(Integer, ForeignKey('departments.dept_no'), primary_key = True)
    
    emp_no_deptemp = relationship('Employee', foreign_keys='Dept_employee.emp_no')
    dept_no_deptemp = relationship('Department', foreign_keys='Dept_employee.dept_no')
            
class Salary (Base):
    __tablename__ = 'salaries'
    __table_args__ = {'extend_existing': True}
    emp_no = Column(Integer, ForeignKey('employees.emp_no'),primary_key = True)
    salary = Column(Integer)
    
    emp_no_salary = relationship('Employee', foreign_keys='Salary.emp_no')

In [ ]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///Employee.db')
conn = engine.connect()

In [ ]:
Base.metadata.tables

In [ ]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
result = engine.execute('SELECT * FROM salaries')
print(result)

In [ ]:
pd.read_sql

In [ ]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)